<a href="https://colab.research.google.com/github/ShanChartrand/NECDataCleaning/blob/main/NEC_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install pandas
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
# Read the North End Connect data file

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import requests

url="https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv"
s=requests.get(url).content
necoriginal=pd.read_csv(io.StringIO(s.decode('utf-8')))

# Display a few rows of the data to ensure the file is connected correctly

display(necoriginal.head(10))



,TEST ID,TEST DATE / UTC HOUR,GEO LATITUDE,GEO LONGITUDE,DOWNLOAD SPEED,UPLOAD SPEED,JITTER,LATENCY,PACKET LOSS,MOBILE TEST,...,CONTRACTED UPLOAD SPEED,CONTRACTED DATA CAP,CONTRACTED MONTHLY COST,SERVICE TYPE,USER SATISFACTION,CONNECTION TYPE,USER COMMENTS,HEATMAP_MUNICIPALITY,HEATMAP_CENSUS_TRACTS,HEATMAP_PROJECT_AREA
0,261,2015-05-23 10:00:13,49.869556,-97.296739,16.22,2.79,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020537.02,NaN
1,284,2015-05-20 10:01:05,49.852738,-97.270058,14.89,2.74,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020522.02,NaN
2,622,2015-05-19 10:00:28,49.887575,-97.140091,9.89,5.64,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020014.00,NaN
3,701,2015-05-24 10:00:08,49.814284,-97.160235,10.62,2.09,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020501.02,NaN
4,707,2015-05-19 10:00:19,49.882706,-97.161829,9.89,2.60,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020016.00,NaN
5,765,2015-05-19 10:00:21,49.897501,-97.136397,10.53,0.70,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020024.00,NaN
6,831,2015-05-16 10:00:30,49.946880,-97.058317,5.79,0.56,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020142.02,NaN
7,972,2015-05-16 10:00:33,49.878963,-97.158439,5.79,0.81,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020015.00,NaN
8,1110,2015-05-19 10:00:23,49.846671,-97.257653,1.88,1.81,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020522.02,NaN
9,1147,2015-05-20 10:01:16,49.937924,-97.118305,10.58,2.76,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winnipeg,6020551.00,NaN


Confirming the number of rows and columns, respectively

In [25]:
necoriginal.shape

(19928, 31)

Having trouble locating/executing csv file in here. Checking to see the working directory

In [26]:
import os
print(os.getcwd())

/content


Checking for duplicates in the TEST ID column

In [27]:
# Count the number of duplicated TEST ID values
num_duplicates = df.duplicated(subset='TEST ID', keep=False).sum()

# Print the number of duplicates
print(f"There are {num_duplicates} rows with duplicated TEST ID values.")

There are 0 rows with duplicated TEST ID values.


Looking for duplicate rows

In [28]:
# Find duplicate rows based on all columns
duplicates = df[df.duplicated()]

# Display the duplicate rows (if any)
if len(duplicates) > 0:
    print(f"Found {len(duplicates)} duplicate rows:")
    print(duplicates)
else:
    print("No duplicate rows found.")

No duplicate rows found.


Checking to see how many missing cells in the CLIENT CITY column

In [29]:
df = pd.read_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv')

missing_cells = df['CLIENT CITY'].isnull().sum()

print(missing_cells)

55


Using Longitude and Latitude to fill in the missing CLIENT CITY cells

In [30]:
import pandas as pd
from geopy.geocoders import Nominatim

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv')

# Create a geolocator object
geolocator = Nominatim(user_agent="my-app")

# Define a function to perform reverse geocoding and return the city name
def reverse_geocode(lat, lon):
    location = geolocator.reverse(f"{lat}, {lon}")
    return location.raw['address']['city'] if 'city' in location.raw['address'] else None

# Fill in missing CLIENT CITY values using reverse geocoding
df['CLIENT CITY'] = df.apply(lambda row: reverse_geocode(row['GEO LATITUDE'], row['GEO LONGITUDE']) if pd.isnull(row['CLIENT CITY']) else row['CLIENT CITY'], axis=1)

# Save the updated DataFrame to a new CSV file
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

#Confirming there are no missing values
missing_cells = df['CLIENT CITY'].isnull().sum()

print(missing_cells)


0


Using CLIENT CITY to fill in missing CLIENT PROVINCE values

In [31]:
df['CLIENT PROVINCE'].fillna(df['CLIENT CITY'].str.split(',').str[-1], inplace=True)

# Save the updated DataFrame to a new CSV file
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

Confirming the CLIENT CITY & CLIENT PROVINCE have no missing values

In [32]:
# Calculate the percentage of missing cells in each column
missing_percentages = df.isna().sum() / len(df) * 100

# Print the result
print(missing_percentages)

TEST ID                       0.000000
TEST DATE / UTC HOUR          0.000000
GEO LATITUDE                  0.000000
GEO LONGITUDE                 0.000000
DOWNLOAD SPEED                0.000000
UPLOAD SPEED                  0.000000
JITTER                       40.395424
LATENCY                      40.395424
PACKET LOSS                  40.395424
MOBILE TEST                   0.000000
ISP                           0.005018
CLIENT IP                     0.000000
CLIENT CITY                   0.000000
CLIENT PROVINCE               0.000000
CLIENT POSTAL CODE            0.130470
AS NUMBER                     0.105379
IPV6 SUPPORTED                0.000000
DNSSEC SUPPORTED              0.000000
TEST SERVER                   0.000000
browser_test_url             34.704938
CONTRACTED DOWNLOAD SPEED    81.413087
CONTRACTED UPLOAD SPEED      88.237656
CONTRACTED DATA CAP          92.377559
CONTRACTED MONTHLY COST      83.611000
SERVICE TYPE                 78.577880
USER SATISFACTION        

In [33]:
!pip install IPWhois

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
from ipwhois import IPWhois

Standardizing all CLIENT CITY names e.g. Winnipeg

In [35]:
import pandas as pd

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv')

# Check for unique values in the CLIENT CITY column
cities = df['CLIENT CITY'].unique()

# Print the unique values
print(cities)

# Clean up the city names
df['CLIENT CITY'] = df['CLIENT CITY'].str.title()  # Capitalize the first letter of each word
df['CLIENT CITY'] = df['CLIENT CITY'].str.strip()  # Remove leading/trailing white spaces

# Check for unique values again
cities_cleaned = df['CLIENT CITY'].unique()

# Save the updated dataframe to a new CSV file
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

# Print the unique cleaned values
print(cities_cleaned)


['Winnipeg' nan 'Gimli' 'Sault Ste. Marie' 'Calgary' 'Moose Jaw' 'Winkler'
 'Ottawa' 'Greater Sudbury' 'North York' 'Edmonton' 'Vancouver' 'Guelph'
 'Winnipeg (Portage & Main)' 'Hamilton' 'Surrey' 'Notre Dame de Lourdes'
 'Winnipeg (Tuxedo Industrials)' 'Barrie' 'Woodstock'
 'Toronto (Etobicoke)' 'Creemore' 'Alliston' 'Calgary (Downtown)'
 'St. Catharines' 'Toronto' 'Selkirk' 'Oak Bay' 'London' 'Kitchener'
 'Wawanesa' 'Cambridge' 'Brantford' 'Nobleton' 'Beauharnois'
 'Portage la Prairie' 'Roland' 'Caledon' 'Waterloo' 'Concord'
 'Mississauga' 'Port Perry' 'Orangeville' 'Stony Mountain' 'Oakville'
 'Brampton' 'Montreal' 'Toronto (Old Toronto)' 'Winnipeg (Downtown)'
 'Angus' 'Thompson' 'Wabowden' 'Winnipeg (Sir John Franklin)'
 'Winnipeg (Earl Grey)' 'Winnipeg (Kildare - Redonda)' 'Miami'
 'Scarborough' 'Carman' 'Saint-Quentin' 'La Salle' 'Oakbank' 'Marathon'
 'Georgetown' 'Port Hope' 'Thorold' 'Old Toronto' 'Blackstock' 'Lindsay'
 'Charlemagne' 'Steinbach' 'Etobicoke' 'Brandon' 'Marchand

Standardizing all CLIENT PROVINCE names e.g. MB